In [22]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timezone

In [26]:
# URL of the SpaceX launch schedule
url = 'https://www.rocketlaunch.live/?filter=spacex'

# Send a GET request to the website
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Find the container with launch information
launches = soup.find_all('div', class_='launch')

# Initialize a list to store launch data
data = []

# Loop through each launch entry and extract details
for launch in launches:
    # Mission Name
    mission_name_tag = launch.find('div', class_='mission_name')
    mission_name = mission_name_tag.get_text(strip=True) if mission_name_tag else None

    # Vehicle
    vehicle_tag = launch.select_one('.rlt-vehicle h4 a')
    vehicle = vehicle_tag.get_text(strip=True) if vehicle_tag else None

    # Company
    company_tag = launch.select_one('.rlt-provider a')
    company = company_tag.get_text(strip=True) if company_tag else None

    # Location
    location_tag = launch.select_one('.rlt-location')
    location = location_tag.get_text(separator=' ', strip=True) if location_tag else None

    # Date and Time
    updated_tag = launch.select_one('.updated.rlt_datetime')
    datetime_tag = launch.select_one('.launch_datetime')

    # Try updated_tag first if  in the future
    if updated_tag:
        updated_dt_str = updated_tag.get_text(strip=True)
        try:
            updated_dt = datetime.fromisoformat(updated_dt_str.replace('Z', '+00:00'))
            if updated_dt > datetime.now(timezone.utc):
                date = updated_dt.date().isoformat()
                time = updated_dt.time().strftime('%H:%M:%S')
            else:
                # Fall back to launch_datetime
                date = datetime_tag['data-sortdate'] if datetime_tag and 'data-sortdate' in datetime_tag.attrs else None
                time = datetime_tag.select_one('.rlt_time')
                time = time.get_text(strip=True) if time else None
        except Exception:
            # If parsing fails, fall back
            date = datetime_tag['data-sortdate'] if datetime_tag and 'data-sortdate' in datetime_tag.attrs else None
            time = datetime_tag.select_one('.rlt_time')
            time = time.get_text(strip=True) if time else None
    else:
        # No updated_tag at all
        date = datetime_tag['data-sortdate'] if datetime_tag and 'data-sortdate' in datetime_tag.attrs else None
        time = datetime_tag.select_one('.rlt_time')
        time = time.get_text(strip=True) if time else None
    
    # Description
    desc_tag = launch.select_one('.launch_desc_and_links p a')
    description = desc_tag.get_text(strip=True) if desc_tag else None

    # Tags
    tag_elements = launch.select('.launch_tag a')
    tags = [tag.get_text(strip=True) for tag in tag_elements]

    # Store all the extracted info
    data.append({
        'mission_name': mission_name,
        'vehicle': vehicle,
        'company': company,
        'location': location,
        'date': date,
        'time': time,
        'description': description,
        'tags': tags
        })

# Create a DataFrame
df = pd.DataFrame(data)

# Save to CSV
df['date'] = pd.to_datetime(df['date'], errors='coerce').dt.date
df.to_csv('spacex_launch_schedule.csv', index=False)
print(df)

                              mission_name                           vehicle  \
0                        Starship Flight 9  Super Heavy / Starship Prototype   
1                                     Ax-4                          Falcon 9   
2                                  Crew-11                          Falcon 9   
3                SDA Tranche 1 Transport B                          Falcon 9   
4                                   MTG-S1                          Falcon 9   
5                                  TRACERS                          Falcon 9   
6                                     IMAP                          Falcon 9   
7                        Griffin Mission 1                      Falcon Heavy   
8            NASA Lunar Gateway PPE & HALO                      Falcon Heavy   
9                           Project Kuiper                          Falcon 9   
10                          Project Kuiper                          Falcon 9   
11                          Project Kuip